# AI-Powered FinTech Chatbot
## Enhancing Customer Service and User Experience

The fintech chatbot project comprises two files that are central to the development of an intelligent chatbot for financial services. 

The first file, 'train_chatbot.ipynb', is responsible for pre-processing and preparing the data for training the chatbot's machine learning model. This includes reading in and tokenizing the intents, patterns, and responses from a JSON file, lemmatizing and cleaning the text data, generating a bag-of-words model, and fitting a neural network to the prepared data. 

The second file, 'chatbot.ipynb', is responsible for using the trained model to predict the intent of incoming user messages and generate appropriate responses. This is accomplished using a combination of natural language processing (NLP) techniques and deep learning, allowing the chatbot to interact with users and provide tailored financial advice and services.

In [74]:
# Import required libraries
import random
import json
import pickle
import numpy as np
import tensorflow as tf

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer

In [57]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [58]:
# Create an instance of the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
# Load the intents file in JSON format
intents = json.loads(open('intents.json').read())

In [60]:
# Create empty lists to hold words and document patterns
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

In [61]:
# Loop through each intent and pattern in the intents file
for intent in intents['intents']:
    for pattern in intent['patterns']:
        
        # Tokenize the pattern into a list of words
        word_list = nltk.word_tokenize(pattern)
        
        # Add the words to the words list
        words.extend(word_list)
        
        # Add the word list and the intent tag to the documents list
        documents.append((word_list, intent['tag']))
        
        # If the intent tag is not already in the classes list, add it
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Print the documents list for debugging
print(documents)

[(['Hi'], 'greeting'), (['Hello'], 'greeting'), (['Hey'], 'greeting'), (['Hi', 'there'], 'greeting'), (['Greetings'], 'greeting'), (['Bye'], 'goodbye'), (['Goodbye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Take', 'care'], 'goodbye'), (['Thanks', ',', 'bye'], 'goodbye'), (['What', 'are', 'your', 'hours', '?'], 'hours'), (['When', 'are', 'you', 'open', '?'], 'hours'), (['What', 'time', 'do', 'you', 'close', '?'], 'hours'), (['What', 'is', 'my', 'account', 'balance', '?'], 'balance'), (['How', 'much', 'money', 'do', 'I', 'have', '?'], 'balance'), (['What', "'s", 'my', 'balance', '?'], 'balance'), (['Can', 'you', 'show', 'me', 'my', 'recent', 'transactions', '?'], 'transactions'), (['What', 'are', 'my', 'latest', 'transactions', '?'], 'transactions'), (['I', 'need', 'to', 'see', 'my', 'transaction', 'history', '.'], 'transactions'), (['What', 'is', 'my', 'available', 'credit', '?'], 'credit'), (['What', 'is', 'my', 'credit', 'limit', '?'], 'credit'), (['How', 'much', 'credit'

In [62]:
# Download the WordNet dataset from NLTK
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\user.DESKTOP-
[nltk_data]     OMQ89VA\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [63]:
# Lemmatize each word in the words list and remove any non-string values and ignore_letters
words = [lemmatizer.lemmatize(word) for word in words if isinstance(word, str) and word not in ignore_letters]

# Sort and remove duplicates from the words list
words = sorted(set(words))

# Sort and remove duplicates from the classes list
classes = sorted(set(classes))

In [65]:
# Save the words and classes lists as binary files
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [66]:
# Create an empty training list and output row
training = []
output_empty = [0] * len(classes)

In [67]:
# Loop through each document in the documents list
for document in documents:
    
    # Create an empty bag list
    bag = []
    
    # Get the list of words from the document and lemmatize each word
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    
    # Loop through each word in the words list and add 1 to the bag list if the word is in the word_patterns list, 0 otherwise
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    
    # Create an output row with all zeros and a 1 in the position of the intent tag
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    
    # Add the bag and output row to the training list
    training.append([bag, output_row])

In [68]:
# Shuffle the training list and convert it to a numpy array
random.shuffle(training)
training = np.array(training, dtype=object)

In [69]:
# Split the training data into input (X) and output (y) variables
train_x = list(training[: , 0])
train_y = list(training[: , 1])

In [70]:
# Define the neural network model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [71]:
# Define a learning rate schedule using Exponential Decay to reduce the learning rate over time
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.96)

# Define the Stochastic Gradient Descent (SGD) optimizer with Nesterov momentum and the learning rate schedule
sgd = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9, nesterov=True)

# Compile the model with categorical crossentropy as the loss function, the SGD optimizer, and accuracy as the metric
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [72]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
9/9 [==============================] - 1s 12ms/step - loss: 2.4734 - accuracy: 0.0976
Epoch 2/200
9/9 [==============================] - 0s 5ms/step - loss: 2.4769 - accuracy: 0.1463
Epoch 3/200
9/9 [==============================] - 0s 5ms/step - loss: 2.4046 - accuracy: 0.1951
Epoch 4/200
9/9 [==============================] - 0s 6ms/step - loss: 2.2981 - accuracy: 0.2439
Epoch 5/200
9/9 [==============================] - 0s 9ms/step - loss: 2.2218 - accuracy: 0.2439
Epoch 6/200
9/9 [==============================] - 0s 8ms/step - loss: 2.2015 - accuracy: 0.2927
Epoch 7/200
9/9 [==============================] - 0s 7ms/step - loss: 2.0657 - accuracy: 0.4390
Epoch 8/200
9/9 [==============================] - 0s 7ms/step - loss: 2.0792 - accuracy: 0.3415
Epoch 9/200
9/9 [==============================] - 0s 6ms/step - loss: 1.9543 - accuracy: 0.3659
Epoch 10/200
9/9 [==============================] - 0s 7ms/step - loss: 1.8437 - accuracy: 0.5122
Epoch 11/200
9/9 [==========

9/9 [==============================] - 0s 4ms/step - loss: 0.1043 - accuracy: 0.9756
Epoch 168/200
9/9 [==============================] - 0s 3ms/step - loss: 0.1631 - accuracy: 0.9512
Epoch 169/200
9/9 [==============================] - 0s 3ms/step - loss: 0.1209 - accuracy: 0.9756
Epoch 170/200
9/9 [==============================] - 0s 3ms/step - loss: 0.1525 - accuracy: 0.9512
Epoch 171/200
9/9 [==============================] - 0s 5ms/step - loss: 0.1154 - accuracy: 0.9756
Epoch 172/200
9/9 [==============================] - 0s 6ms/step - loss: 0.0992 - accuracy: 0.9756
Epoch 173/200
9/9 [==============================] - 0s 6ms/step - loss: 0.1122 - accuracy: 0.9756
Epoch 174/200
9/9 [==============================] - 0s 4ms/step - loss: 0.1326 - accuracy: 0.9512
Epoch 175/200
9/9 [==============================] - 0s 3ms/step - loss: 0.1860 - accuracy: 0.9512
Epoch 176/200
9/9 [==============================] - 0s 4ms/step - loss: 0.1127 - accuracy: 0.9756
Epoch 177/200
9/9 [=====

In [73]:
# Save the model to a file named 'chatbotmodel.h5' along with the training history
model.save('chatbotmodel.h5', hist)